---
title: "Lab 4"
format: 
  html:
    theme: lux
---

# Lab 4: Coffee Lovers Unite
# Coffee Lovers Unite!
If caffeine is one of the most popular drugs, then coffee is likely one of the most popular delivery systems for it. Aside from caffeine, people enjoy the wonderful variety of coffee-related drinks. Let’s do a rough investigation of the “market share” by two of the top coffee chains in the United States!

World Population Review provides some great data on store locations and chain prevalence. Check out this page for the Starbucks Coffee locations in the United States. Notice that this page only really gives the name of the state and the number of locations in that state.

## Scrape the Location Counts
### 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:

- Starbucks

- Dunkin’ Donuts



In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Starbucks data extraction
response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")
sb_soup = BeautifulSoup(response.content, "html.parser")
sb_tables = sb_soup.find_all("table", class_="wpr-table")[0]
sb_df = pd.read_html(str(sb_tables))[0]

sb_rows = []
states_list = []

for th in sb_tables.find_all("th")[4:]:
    states = th.get_text(strip=True)
    states_list.append(states)

for i, tr in enumerate(sb_tables.find_all("tr")[1:]):
    state = states_list[i]
    cells = tr.find_all("td")

    stores2023_tag = cells[0].find("a") or cells[0]
    stores2023 = stores2023_tag.get_text(strip=True)

    stores2024_tag = cells[1].find("a") or cells[1]
    stores2024 = stores2024_tag.get_text(strip=True)

    stores2021_tag = cells[2].find("a") or cells[2]
    stores2021 = stores2021_tag.get_text(strip=True)

    sb_rows.append({
        "state": state,
        "2023": stores2023,
        "2021": stores2021,
        "2024": stores2024
    })
sb = pd.DataFrame(sb_rows)

# Dunkin' Donuts data extraction
response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")
soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", class_="wpr-table")

dd_rows = []
states_list = []
for th in table.find_all("th")[3:]:
    states = th.get_text(strip=True)
    states_list.append(states)

for i, tr in enumerate(table.find_all("tr")[1:]):
    state = states_list[i]
    cells = tr.find_all("td")

    stores2024_tag = cells[0].find("a") or cells[0]
    stores2024 = stores2024_tag.get_text(strip=True)

    stores2023_tag = cells[1].find("a") or cells[1]
    stores2023 = stores2023_tag.get_text(strip=True)

    dd_rows.append({
        "state": state,
        "2023": stores2023,
        "2024": stores2024
    })

dd = pd.DataFrame(dd_rows)



<ipython-input-1-5f73daee6a65>:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sb_df = pd.read_html(str(sb_tables))[0]


### 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [2]:
# Data transformation
sb_clean = sb.melt(id_vars="state", var_name="year", value_name="stores")
sb_clean["name"] = "Starbucks"

dd_clean = dd.melt(id_vars=["state"], var_name="year", value_name="stores")
dd_clean["name"] = "Dunkin' Donuts"

merged_coffee = sb_clean.merge(dd_clean, on=["state", "year", "stores", "name"], how="outer")

# Convert 'year' to integer and filter to keep only 2023 and 2024
merged_coffee["year"] = merged_coffee["year"].astype(int)
merged_coffee = merged_coffee[merged_coffee["year"].isin([2023, 2024])]

merged_coffee.head()


,state,year,stores,name
1,Alabama,2023,59,Dunkin' Donuts
2,Alabama,2023,85,Starbucks
3,Alabama,2024,69,Dunkin' Donuts
4,Alabama,2024,99,Starbucks
6,Alaska,2023,0,Dunkin' Donuts



## Supplemental Data

### 3. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

In [3]:
response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")
soup = BeautifulSoup(response.content, "html.parser")

tables = soup.find_all("table", class_ = "wikitable")
table = tables[0]
rows = []

for tr in table.find_all("tr")[1:]:
    cells = tr.find_all("td")

    state_t = cells[2].find("a") or cells[2]
    state = state_t.get_text(strip = True)

    pop_t = cells[3].find("a") or cells[3]
    population = pop_t.get_text(strip = True)

    rows.append({
        "state" : state,
        "population" : population
    })

population = pd.DataFrame(rows)
merged_population = merged_coffee.merge(population, on = "state", how = "left")
merged_population.head()

,state,year,stores,name,population
0,Alabama,2023,59,Dunkin' Donuts,"5,024,279"
1,Alabama,2023,85,Starbucks,"5,024,279"
2,Alabama,2024,69,Dunkin' Donuts,"5,024,279"
3,Alabama,2024,99,Starbucks,"5,024,279"
4,Alaska,2023,0,Dunkin' Donuts,"733,391"



### 4. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

In [4]:
# Starbucks revenue
sb_revenue = pd.DataFrame({
    "year": [2023, 2024],
    "revenue": [29.5, 36.2],
    "name": ["Starbucks"] * 2
})

# Dunkin' Donuts revenue
dd_revenue = pd.DataFrame({
    "year": [2023, 2024],
    "revenue": [1.4, 1.6],
    "name": ["Dunkin' Donuts"] * 2
})

merged_revenue = pd.concat([sb_revenue, dd_revenue], ignore_index=True)
rev_coffee = merged_coffee.merge(merged_revenue, on=["name", "year"], how="left")

rev_coffee.head()


,state,year,stores,name,revenue
0,Alabama,2023,59,Dunkin' Donuts,1.4
1,Alabama,2023,85,Starbucks,29.5
2,Alabama,2024,69,Dunkin' Donuts,1.6
3,Alabama,2024,99,Starbucks,36.2
4,Alaska,2023,0,Dunkin' Donuts,1.4





### 5. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [5]:
# Define regions with states
us_regions = {
    "Northeast": [
        "Connecticut", "Maine", "Massachusetts", "New Hampshire",
        "Rhode Island", "Vermont", "New Jersey", "New York", "Pennsylvania"
    ],
    "Midwest": [
        "Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin",
        "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska",
        "North Dakota", "South Dakota"
    ],
    "South": [
        "Delaware", "Florida", "Georgia", "Maryland", "North Carolina",
        "South Carolina", "Virginia", "Washington, D.C.", "West Virginia",
        "Alabama", "Kentucky", "Mississippi", "Tennessee",
        "Arkansas", "Louisiana", "Oklahoma", "Texas"
    ],
    "West": [
        "Arizona", "Colorado", "Idaho", "Montana", "Nevada",
        "New Mexico", "Utah", "Wyoming", "Alaska", "California",
        "Hawaii", "Oregon", "Washington"
    ]
}

# Function to get the region of a state
def get_region(state):
    for region, states in us_regions.items():
        if state in states:
            return region
    return "No region"  # For territories or unlisted states

# Test the function with an example state
state_name = "Texas"
region = get_region(state_name)
print(f"{state_name} is in the {region} region.")


Texas is in the South region.


## Analyze
### 6. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):

### Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.

West and Midwest have way more Starbucks than Dunkin. Northeast has more Dunkin than Starbucks. South is a lot closer in distribution.

### How much revenue does each company have in each year in 2023 and 2024 per store?

Starbucks has more revenue per store in both 2023 and 2024, and it increases quite a bit in 2024.

### Does the distribution of each chain’s stores match population distribution, by both state/region?

No, the distribution doesn't match the population distribution. Starbucks is more prevalent in some states while Dunkin is more prevalent in other states. Regions are interesting because it seems the South and West have less coffee stores than the other regions per 100,000 people.


In [6]:
# Ensure revenue and population are properly merged
# Re-run the merge and assign to 'merged_population'
merged_population = rev_coffee.merge(population, on="state", how="left")

# Check if 'revenue' column is in the dataframe
print("Columns in merged_population:", merged_population.columns)

# Proceed with the region assignment and calculations only if 'revenue' is present
if 'revenue' in merged_population.columns:
    # Convert population column to integers for calculations
    merged_population["population"] = merged_population["population"].str.replace(",", "").astype(int)

    # Convert 'stores' column to numeric, handling errors
    merged_population["stores"] = pd.to_numeric(merged_population["stores"], errors="coerce")
    # errors='coerce' will set invalid parsing as NaN

    # Define region assignment function
    def assign_region(state):
        if state in us_regions["Northeast"]:
            return "Northeast"
        elif state in us_regions["Midwest"]:
            return "Midwest"
        elif state in us_regions["South"]:
            return "South"
        elif state in us_regions["West"]:
            return "West"
        else:
            return "No region"

    # Apply region assignment
    merged_population["region"] = merged_population["state"].apply(assign_region)

    # ... (rest of your code remains the same) ...

    # Calculate store counts by state and region
    store_counts_by_state = merged_population.groupby(["state", "name"])["stores"].sum().reset_index()
    store_counts_by_region = merged_population.groupby(["region", "name"])["stores"].sum().reset_index()

    # Compare store distribution to population by state and region
    merged_population["stores_per_100k"] = merged_population["stores"] / (merged_population["population"] / 100000)
    stores_vs_population_state = merged_population.groupby(["state", "name"]).agg({
        "stores": "sum",
        "population": "mean",
        "stores_per_100k": "mean"
    }).reset_index()

    stores_vs_population_region = merged_population.groupby(["region", "name"]).agg({
        "stores": "sum",
        "population": "sum"
    }).reset_index()
    stores_vs_population_region["stores_per_100k"] = stores_vs_population_region["stores"] / (stores_vs_population_region["population"] / 100000)




    # Display summaries for analyses
    print("Store Counts by Region:\n", store_counts_by_region)
    print("\nStores per 100,000 People by State:\n", stores_vs_population_state)
    print("\nStores per 100,000 People by Region:\n", stores_vs_population_region)
else:
    print("Error: 'revenue' column is missing in merged_population.")

# Group data by year and company name to get total stores and total revenue per year
yearly_totals = merged_population.groupby(["year", "name"]).agg({
    "stores": "sum",  # Total stores across all states for each year and company
    "revenue": "sum"  # Total revenue for each year and company
}).reset_index()

# Calculate revenue per store for each year and company
yearly_totals["revenue_per_store"] = yearly_totals["revenue"] / yearly_totals["stores"]

# Display the results
print("Revenue per Year per Company by Total Stores:\n", yearly_totals)


Columns in merged_population: Index(['state', 'year', 'stores', 'name', 'revenue', 'population'], dtype='object')
Store Counts by Region:
       region            name  stores
0    Midwest  Dunkin' Donuts  2877.0
1    Midwest       Starbucks  5127.0
2  No region  Dunkin' Donuts    19.0
3  No region       Starbucks    91.0
4  Northeast  Dunkin' Donuts  5148.0
5  Northeast       Starbucks  3673.0
6      South  Dunkin' Donuts  5294.0
7      South       Starbucks  6416.0
8       West  Dunkin' Donuts   734.0
9       West       Starbucks  5749.0

Stores per 100,000 People by State:
              state            name  stores  population  stores_per_100k
0          Alabama  Dunkin' Donuts   128.0   5024279.0         1.273815
1          Alabama       Starbucks   184.0   5024279.0         1.831109
2           Alaska  Dunkin' Donuts     0.0    733391.0         0.000000
3           Alaska       Starbucks    98.0    733391.0         6.681293
4          Arizona  Dunkin' Donuts   212.0   7151502.0  





## Automate
7. Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should

- Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)

- Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)

- Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.#

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_store_data(url):
    # Extract chain name from the URL (assumes format like ".../starbucks-stores-by-state")
    chain_name = re.findall(r'/([^/]+)-by-state', url)
    chain_name = chain_name[0].replace("-", " ").title() if chain_name else "Unknown Chain"

    # Request and parse the HTML page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Locate and read the table data into a DataFrame
    table = soup.find("table")
    df = pd.read_html(str(table))[0]

    # Gather state names
    states_list = [th.get_text(strip=True) for th in table.find_all("th")[3:]]

    # Prepare rows for the DataFrame
    rows = []
    for i, tr in enumerate(table.find_all("tr")[1:]):
        state = states_list[i]
        cells = tr.find_all("td")

        # Adjust columns to match expected layout
        year_data = {f"{2024-i}": cells[i].get_text(strip=True) for i in range(len(cells))}
        row = {"state": state, **year_data}
        rows.append(row)

    # Convert to DataFrame
    store_df = pd.DataFrame(rows)
    store_df = store_df.melt(id_vars=["state"], var_name="year", value_name="stores")
    store_df["name"] = chain_name

    # Scrape population data from Wikipedia
    pop_url = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"
    pop_response = requests.get(pop_url)
    pop_soup = BeautifulSoup(pop_response.content, "html.parser")
    pop_table = pop_soup.find_all("table", class_="wikitable")[0]

    pop_rows = []
    for tr in pop_table.find_all("tr")[1:]:
        cells = tr.find_all("td")
        state_t = cells[2].find("a") or cells[2]
        state = state_t.get_text(strip=True)
        pop_t = cells[3].find("a") or cells[3]
        population = pop_t.get_text(strip=True)

        pop_rows.append({"state": state, "population": population})

    population_df = pd.DataFrame(pop_rows)

    # Merge store data with population data
    merged_df = store_df.merge(population_df, on="state", how="left")

    # Clean up the year and store columns
    merged_df["year"] = merged_df["year"].astype(int)
    merged_df["stores"] = pd.to_numeric(merged_df["stores"], errors="coerce")
    merged_df.dropna(subset=["stores"], inplace=True)  # Remove rows where store count couldn't be parsed

    return merged_df

# Test the function on a URL (other than Starbucks or Dunkin’ Donuts)
url = "https://worldpopulationreview.com/state-rankings/aldi-by-state"
result = scrape_store_data(url)
print(result.head())


<ipython-input-7-9776103868c2>:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


          state  year  stores  name  population
0       Florida  2024   218.0  Aldi  21,538,187
1      Illinois  2024   216.0  Aldi  12,812,508
2          Ohio  2024   164.0  Aldi  11,799,448
3  Pennsylvania  2024   158.0  Aldi  13,002,700
4      New York  2024   131.0  Aldi  20,201,249


Code assisted by Gemini and ChatGPT